In [4]:
pip install torch_geometric

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 6.2 MB/s eta 0:00:00


In [5]:
import math
import numpy as np
import random
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.parameter import Parameter
from torch import Tensor
from sklearn.metrics import roc_auc_score

In [6]:

@torch.no_grad()
def accuracy(pr_logits, gt_labels):
    return (pr_logits.argmax(dim=-1) == gt_labels).float().mean().item()

@torch.no_grad()
def roc_auc(pr_logits, gt_labels):
    return roc_auc_score(gt_labels.cpu().numpy(), pr_logits[:, 1].cpu().numpy())

from torch_geometric.data import Data

def zero_in_degree_removal(data):
    edge_index = data.edge_index
    # keep only with non-zero incoming edges
    valid_ids = torch.unique(edge_index[1])
    node_mask = torch.zeros(len(data.y), dtype=torch.bool)
    node_mask[valid_ids] = True
    valid_mask = edge_index[0].clone().apply_(lambda x: x in valid_ids).bool()
    valid_edges = torch.masked_select(data.edge_index, valid_mask).view(2, -1)
    return Data(
        x=data.x,
        y=data.y,
        edge_index=valid_edges,
        train_mask=(data.train_mask & node_mask[:, None]),
        val_mask=(data.val_mask & node_mask[:, None]),
        test_mask=(data.test_mask & node_mask[:, None]),
    )

In [8]:
__all__ = ['train_step', 'val_step']


def train_step(
    model,
    optimizer,
    labels,
    list_mat,
    mask,
    metric = accuracy,
    device: str = 'cpu'
):
    model.train()
    optimizer.zero_grad()
    output = model(list_mat)
    loss_train = F.cross_entropy(output[mask], labels[mask].to(device))
    acc_train = metric(output[mask], labels[mask].to(device))
    loss_train.backward()
    optimizer.step()
    return loss_train, acc_train


def val_step(
    model,
    labels,
    list_mat,
    mask,
    metric = accuracy,
    device: str = 'cpu'
):
    model.eval()
    with torch.no_grad():
        output = model(list_mat)
        loss_val = F.cross_entropy(output[mask], labels[mask].to(device))
        acc_val = metric(output[mask], labels[mask].to(device))
        return loss_val, acc_val

In [9]:


__all__ = [
    "FSGNN",
    "FSGNN_Large"
]


class FSGNN(nn.Module):

    def __init__(
        self,
        nfeat,
        nlayers,
        nhidden,
        nclass,
        dropout,
        layer_norm: bool = False,
    ):
        super(FSGNN, self).__init__()
        self.fc2 = nn.Linear(nhidden * nlayers,nclass)
        self.dropout = dropout
        self.act_fn = nn.ReLU()
        self.fc1 = nn.ModuleList([nn.Linear(nfeat,int(nhidden)) for _ in range(nlayers)])
        self.att = nn.Parameter(torch.ones(nlayers))
        self.sm = nn.Softmax(dim=0)
        self.layer_norm = layer_norm

    def forward(self, list_mat):
        mask = self.sm(self.att)
        list_out = list()
        for ind, mat in enumerate(list_mat):
            tmp_out = self.fc1[ind](mat)
            if self.layer_norm == True:
                tmp_out = F.normalize(tmp_out, p=2, dim=1)
            tmp_out = torch.mul(mask[ind],tmp_out)
            list_out.append(tmp_out)

        final_mat = torch.cat(list_out, dim=1)
        out = self.act_fn(final_mat)
        out = F.dropout(out,self.dropout,training=self.training)
        out = self.fc2(out)

        return out


class FSGNN_Large(nn.Module):

    def __init__(
        self,
        nfeat,
        nlayers,
        nhidden,
        nclass,
        dp1,
        dp2,
        layer_norm: bool = True
    ):
        super(FSGNN_Large,self).__init__()
        self.wt1 = nn.ModuleList([nn.Linear(nfeat,int(nhidden)) for _ in range(nlayers)])
        self.fc2 = nn.Linear(nhidden*nlayers,nhidden)
        self.fc3 = nn.Linear(nhidden,nclass)
        self.dropout1 = dp1
        self.dropout2 = dp2
        self.act_fn = nn.ReLU()
        self.att = nn.Parameter(torch.ones(nlayers))
        self.sm = nn.Softmax(dim=0)
        self.layer_norm = layer_norm


    def forward(self, list_adj, st=0, end=0):

        mask = self.sm(self.att)
        mask = torch.mul(len(list_adj),mask)

        list_out = list()
        for ind, mat in enumerate(list_adj):
            mat = mat[st:end,:].cuda()
            tmp_out = self.wt1[ind](mat)
            if self.layer_norm == True:
                tmp_out = F.normalize(tmp_out,p=2,dim=1)
            tmp_out = torch.mul(mask[ind],tmp_out)

            list_out.append(tmp_out)

        final_mat = torch.cat(list_out, dim=1)

        out = self.act_fn(final_mat)
        out = F.dropout(out,self.dropout1,training=self.training)
        out = self.fc2(out)

        out = self.act_fn(out)
        out = F.dropout(out,self.dropout2,training=self.training)
        out = self.fc3(out)

        return out

In [11]:
from copy import deepcopy
from torch_geometric.utils import to_dense_adj, add_self_loops

from torch_geometric.data import Data
from torch_geometric.datasets import Actor, WikipediaNetwork, WebKB


DATASET_LIST = [
    'squirrel_directed', 'chameleon_directed', 'chameleon_filtered', 'squirrel_filtered',
    'squirrel_filtered_directed', 'chameleon_filtered_directed', 'wisconsin',
    'roman_empire', 'minesweeper', 'questions', 'amazon_ratings', 'tolokers'
]


def load_custom_data(data_path, to_undirected: bool = True):
    npz_data = np.load(data_path)
    # convert graph to bidirectional
    if to_undirected:
        edges = np.concatenate((npz_data['edges'], npz_data['edges'][:, ::-1]), axis=0)
    else:
        edges = npz_data['edges']

    data = Data(
        x=torch.from_numpy(npz_data['node_features']),
        y=torch.from_numpy(npz_data['node_labels']),
        edge_index=torch.from_numpy(edges).T,
        train_mask=torch.from_numpy(npz_data['train_masks']).T,
        val_mask=torch.from_numpy(npz_data['val_masks']).T,
        test_mask=torch.from_numpy(npz_data['test_masks']).T,
    )
    return [data]


def get_dataset(dataset):
    if dataset == 'actor':
        return Actor(root='./pyg_data/actor')
    if dataset == 'squirrel':
        return WikipediaNetwork(root='./pyg_data', name='squirrel')
    if dataset == 'chameleon':
        return WikipediaNetwork(root='./pyg_data', name='chameleon')
    if dataset in ['cornell', 'texas', 'wisconsin']:
        return WebKB(root='./pyg_data', name=dataset)
    if dataset in DATASET_LIST:
        if dataset == 'chameleon_directed' or 'chameleon_filtered_directed' or 'squirrel_directed' or 'squirrel_filtered_directed':
          return load_custom_data(
              f'./{dataset}.npz',
              False
            )
        else:
          return load_custom_data(
              f'./{dataset}.npz',
              True
          )
    raise ValueError("Unknown dataset")


def parse_args():
    # Training settings
    args = dict()
    args['seed'] = 42
    args['steps'] = 1500
    args['log_freq'] = 100
    args['num_layers'] = 3
    args['hidden_dim'] = 64
    args['patience'] =100
    args['layer_norm'] = 1
    args['lr'] = 0.001
    args['weight_decay'] = 1e-5
    args['dropout'] = 0.5
    args['feat_type'] = "all"
    print(args)
    return args


def run_on_split(
    features,
    labels,
    list_mat,
    train_mask,
    val_mask,
    test_mask,
    args
):
    num_features = features.shape[1]
    num_labels = len(torch.unique(labels))

    model = FSGNN(
        nfeat=num_features,
        nlayers=len(list_mat),
        nhidden=args['hidden_dim'],
        nclass=num_labels,
        dropout=args['dropout'],
        layer_norm=args['layer_norm'],
    ).to(device)

    optimizer = torch.optim.Adam(model.parameters(), lr=args['lr'], weight_decay=args['weight_decay'])

    metric = accuracy if len(torch.unique(labels)) > 2 else roc_auc

    best = -torch.inf
    best_params = None
    bad_counter = 0
    for step in range(args['steps']):
        loss_train, metric_train = train_step(model, optimizer, labels, list_mat, train_mask, metric, device=device)
        loss_val, metric_val = val_step(model, labels, list_mat, val_mask, metric, device=device)

        if step % args['log_freq'] == 0:
            print(f'Train metric {metric_train:.3f} / Val acc {metric_val:.3f}')

        if metric_val > best:
            best = metric_val
            bad_counter = 0
            best_params = deepcopy(model.state_dict())
        else:
            bad_counter += 1

        if bad_counter == args['patience']:
            break
    # load best params
    model.load_state_dict(best_params)
    loss_test, metric_test = val_step(model, labels, list_mat, test_mask, metric, device=device)
    # return test accuracy
    return metric_test


if __name__ == '__main__':
    args = parse_args()
    # fix seeds
    random.seed(args['seed'])
    np.random.seed(args['seed'])
    torch.manual_seed(args['seed'])
    # get device
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    # get dataset
    dataset = 'wisconsin'
    dataset = get_dataset(dataset)
    data = dataset[0].to(device)

    features = data.x
    labels = data.y
    # get adjacency matrix and its powers
    adj = to_dense_adj(data.edge_index)[0]
    adj_i = to_dense_adj(add_self_loops(data.edge_index)[0])[0]

    list_mat = []
    list_mat.append(features)
    no_loop_mat = features
    loop_mat = features

    for ii in range(args['num_layers']):
        no_loop_mat = torch.spmm(adj, no_loop_mat)
        loop_mat = torch.spmm(adj_i, loop_mat)
        list_mat.append(no_loop_mat)
        list_mat.append(loop_mat)

    # Select X and self-looped features
    if args['feat_type'] == "homophily":
        select_idx = [0] + [2 * ll for ll in range(1, args.num_layers + 1)]
        list_mat = [list_mat[ll] for ll in select_idx]

    #Select X and no-loop features
    elif args['feat_type'] == "heterophily":
        select_idx = [0] + [2*ll - 1 for ll in range(1, args.num_layers + 1)]
        list_mat = [list_mat[ll] for ll in select_idx]

    num_splits = data.train_mask.shape[1]
    test_accs = []
    for i in range(num_splits):
        print(f'Split [{i+1}/{num_splits}]')
        train_mask, val_mask, test_mask = \
            data.train_mask[:, i], data.val_mask[:, i], data.test_mask[:, i]
        test_acc = run_on_split(features, labels, list_mat, train_mask, val_mask, test_mask, args)
        print(f'Test accuracy {test_acc:.3f}')
        test_accs.append(100 * test_acc)

    print(f'Test accuracy {np.mean(test_accs):.2f} +- {np.std(test_accs):.2f}')

{'seed': 42, 'steps': 1500, 'log_freq': 100, 'num_layers': 3, 'hidden_dim': 64, 'patience': 100, 'layer_norm': 1, 'lr': 0.001, 'weight_decay': 1e-05, 'dropout': 0.5, 'feat_type': 'all'}


Processing...
Done!


Split [1/10]
Train metric 0.142 / Val acc 0.525
Train metric 0.900 / Val acc 0.675
Test accuracy 0.588
Split [2/10]
Train metric 0.008 / Val acc 0.113
Train metric 0.917 / Val acc 0.613
Test accuracy 0.784
Split [3/10]
Train metric 0.392 / Val acc 0.488
Train metric 0.825 / Val acc 0.562
Test accuracy 0.549
Split [4/10]
Train metric 0.167 / Val acc 0.300
Train metric 0.875 / Val acc 0.637
Train metric 0.933 / Val acc 0.675
Test accuracy 0.667
Split [5/10]
Train metric 0.225 / Val acc 0.575
Train metric 0.892 / Val acc 0.588
Test accuracy 0.627
Split [6/10]
Train metric 0.142 / Val acc 0.287
Train metric 0.900 / Val acc 0.550
Test accuracy 0.706
Split [7/10]
Train metric 0.042 / Val acc 0.175
Train metric 0.900 / Val acc 0.650
Test accuracy 0.686
Split [8/10]
Train metric 0.092 / Val acc 0.263
Train metric 0.892 / Val acc 0.700
Test accuracy 0.706
Split [9/10]
Train metric 0.050 / Val acc 0.200
Train metric 0.875 / Val acc 0.588
Test accuracy 0.549
Split [10/10]
Train metric 0.233 / Val